In [ ]:
from onnxruntime.transformers.convert_generation import parse_arguments, GenerationType, convert_generation_model
from models.gpt2.convert_to_onnx import main as convert_gpt2_to_onnx

In [ ]:
from pathlib import Path

model_path = Path.cwd().joinpath('models')
model_id = 'bio-gpt-qa'
model_path = model_path.joinpath(model_id)

In [ ]:
onnx_path = Path.cwd().joinpath('models_repository',
                                "generator", "generator_model", "1", )

In [ ]:
onnx_model_with_past = onnx_path.joinpath('bio-gpt-model-with-past')

In [ ]:
from argparse import Namespace

In [ ]:
arguments = Namespace(
    model_name_or_path=model_path.__str__(),
    output=onnx_model_with_past.parent.joinpath(
        "biogpt-model-with-past-and-beam.onnx").__str__(),
    model_type="gpt2",
    num_beams="5",
    temperature="0.25",
    model_class="BioGptModel"
)

In [ ]:
onnx_model_with_past.joinpath("model.onnx").exists()

In [ ]:
arguments_list = []

for key, value in vars(arguments).items():
    arguments_list.append(f"--{key}")
    arguments_list.append(value)

In [ ]:
arguments_list

In [ ]:
args = parse_arguments(arguments_list)

In [ ]:
convert_generation_model(args=args, generation_type=GenerationType.BEAMSEARCH)

Testing the model for the inference session

In [ ]:
model_with_beam = onnx_model_with_past.parent.joinpath(
        "biogpt-model-with-past-and-beam.onnx").__str__()

In [ ]:
from onnxruntime import InferenceSession

In [ ]:
import torch

In [ ]:
inference_session = InferenceSession(model_with_beam, providers=['CPUExecutionProvider'])

Yeah, it worked.. 

Generating inputs

In [ ]:
from typing import Sequence

In [ ]:
from transformers import AutoTokenizer, AutoConfig

In [ ]:
EXAMPLE_Text = ["what is the cause of covid-19?"]


def get_tokenizer(model_name_or_path: str, cache_dir: str):
    tokenizer = AutoTokenizer.from_pretrained(
        model_name_or_path, cache_dir=cache_dir)
    tokenizer.padding_side = "left"
    tokenizer.pad_token = tokenizer.eos_token
    return tokenizer

In [ ]:
model_path

In [ ]:
model_config = AutoConfig.from_pretrained(model_path.__str__())

In [ ]:
tokenizer = get_tokenizer(model_path, model_path)
encodings_dict = tokenizer.batch_encode_plus(EXAMPLE_Text, padding=True)

input_ids = torch.tensor(encodings_dict["input_ids"], dtype=torch.int32)

In [ ]:
input_ids

In [ ]:
empty_past

In [ ]:
import numpy as np

In [ ]:
ort_inputs = {
    "input_ids": np.ascontiguousarray(input_ids.cpu().numpy(), dtype=np.int32),
    "max_length": np.array([512], dtype=np.int32),
    "min_length": np.array([128], dtype=np.int32),
    "num_beams": np.array([5], dtype=np.int32),
    "num_return_sequences": np.array([1], dtype=np.int32),
    "length_penalty": np.array([1.0], dtype=np.float32),
    "repetition_penalty": np.array([1.4], dtype=np.float32),
}

In [ ]:
outputs = inference_session.run(None, ort_inputs)

In [ ]:
output_indices = outputs[0].squeeze()

[autoreload of traitlets.traitlets failed: Traceback (most recent call last):
  File "/Users/esp.py/Projects/Personal/end-to-end-rag/.venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/Users/esp.py/Projects/Personal/end-to-end-rag/.venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/esp.py/Projects/Personal/end-to-end-rag/.venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/Users/esp.py/Projects/Personal/end-to-end-rag/.venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 365, in update_class
    update_instances(old, new)
  File "/Users/esp.py/Projects/Personal/end-to-end-rag/.venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 323, in update_instances
    object.__setattr__(ref, "__class__"

TypeError: isinstance() arg 2 must be a type, a tuple of types, or a union

In [ ]:
tokenizer = get_tokenizer(model_path.__str__(), model_path.__str__())

In [ ]:
tokenizer.decode(output_indices, skip_special_tokens=True)

the code seems to be working, we can continue with the deployment and write the blogpost.

Need to come back and test the triton inputs and outputs..